<table align="left">
  <td>
    <a href="https://tinyurl.com/2khrsfbc" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table>

# Natural Language Processing with TensorFlow

El objetivo principal del [procesamiento de lenguaje natural](https://tinyurl.com/y68tj6v6) (NLP) es analizar, comprender y derivar información valiosa de texto. Las aplicaciones de NLP pueden clasificarse en dos categorías principales:

- Texto (Como el que se encuentra en un email, un tweet, un documento, etc.)
- Audio (Como el que se encuentra en una grabación de voz, una canción, etc.)

Por ejemplo, si tuviéramos un conjunto de datos de texto que contuviera reseñas de películas, podríamos usar NLP para identificar el sentimiento detrás de cada reseña (por ejemplo, positivo o negativo). También podríamos usar NLP para identificar el tema de cada reseña (por ejemplo, romance, acción, etc.). Si quisieramos construir un identificador de spam, podríamos usar NLP para identificar palabras clave en un correo electrónico que podrían indicar que es spam.

En este notebook, se trabará sobre lo siguiente:

- Descargar y explorar un conjunto de datos de texto
- Visualizar datos de texto
- Convertir texto en números usando tokenización
- Convertir texto tokenizado en vectores usando codificación embebida
- Modelar un dataset de texto
- Predecir el sentimiento de una oración
    - Iniciar con una referencia (TF-IDF)
    - Construir diferentes modelos de texto de deep learning
        - Dense, LSTM, GRU, Conv1D, Transfer Learning
- Comparar el desempeño de cada modelo
- Combinar los en un ensamble
- Guardar y cargar un modelo entrenado
- Encontrar las peores predicciones.

## 1. Descargar y explorar un dataset de texto

Se utilizará el dataset [Real or Not?](https://www.kaggle.com/c/nlp-getting-started/data) que contiene Tweets acerca de desastres naturales. El objetivo es predecir si un tweet es acerca de un desastre real o no.

- Un Tweet **Real** es acerca de un desastre real, por ejemplo:
> Jetstar and Virgin forced to cancel Bali flights again because of ash from Mount Raung volcano

- Un Tweet **No Real** no es acerca de un desastre real pueden ser sobre cualquier cosa, por ejemplo:
> 'Education is the most powerful weapon which you can use to change the world.' Nelson #Mandela #quote

Al descomprimir el dataset se obtendran tres archivos:

- `sample_submission.csv` - un archivo de muestra de la estructura de envío de predicciones en las competencias de Kaggle.
- `train.csv` - el conjunto de datos de entrenamiento que contiene los tweets reales y no reales de desastres etiquetados.
- `test.csv` - el conjunto de datos de prueba que contiene los tweets reales y no reales que se deben predecir.


### 1.1 Visualizar un dataset de texto

Una vez que se cuenta con un dataset para trabajar, es una buena idea visualizarlo para ver si hay algo que se pueda entender de él. El dataset se encuentra en archivos `.csv`, una forma sencilla de hacerlos legibles es usando la función `pd.read_csv()` de pandas.

In [1]:
# Turn .csv files into pandas DataFrame's
import pandas as pd
train_df = pd.read_csv("nlp_real_not_dataset/train.csv")
test_df = pd.read_csv("nlp_real_not_dataset/test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


los datos descargados seguramente se encuentran mezclados. Pero para asegurarnos es una buena idea volverlos a mezclar.

In [2]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42) # shuffle with random_state=42 for reproducibility
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


Los datos de entrenamiento contienen la columna `target` que indica si un tweet es real o no real. Esta columna será la etiqueta que se usará para entrenar el modelo. El dataset de prueba no contiene esta columna, ya que es lo que se debe predecir.

La estructura que se quiere implementar es la siguiente:

> Inputs (text column) -> Machine Learning Algorithm -> Outputs (target column)

<img src="https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/images/08-text-classification-inputs-and-outputs.png" alt="nlp" border="0" width = 800>

In [3]:
# The test data doesn't have a target (that's what we'd try to predict)
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [4]:
# How many examples of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

Dado que se cuenta con dos valores objetivo (0 y 1), se trata de un problema de clasificación binaria. El dataset se encuentra desbalanceado, con un 60% de tweets de clase 0 y un 40% de tweets de clase 1.

donde: 

- 1 = Un tweet es acerca de un desastre real
- 0 = Un tweet no es acerca de un desastre real